En este archivo, voy a desarrollar la función que se me pide en el segundo endpoint, que es la que se muestra a continuación:

![alt text](../img/endpoint_2.png "Title")

Importación de las librerías necesarias para trabajar

In [20]:
import pandas as pd

Lectura de los conjuntos de datos necesarios necesarios y breve vistazo de la cabecera de los mismos

In [21]:
user_reviews = pd.read_parquet("../EDA/user_reviews_complete.parquet")
user_reviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637


In [22]:
user_items = pd.read_parquet("../EDA/user_items_complete.parquet")
user_items.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6,0,76561197970982479
1,20,Team Fortress Classic,0,0,76561197970982479
2,30,Day of Defeat,7,0,76561197970982479
3,40,Deathmatch Classic,0,0,76561197970982479
4,50,Half-Life: Opposing Force,0,0,76561197970982479


In [23]:
steam_games = pd.read_parquet("../Datasets/steam_games_post_EDA.parquet")
steam_games.head()

,id,app_name,developer,price,release_year,categorical
88310,761140,Lost Summoner Kitty,Kotoshiro,4.99,2018,"[Action, Casual, Indie, Simulation, Strategy, ..."
88311,643980,Ironbound,Secret Level SRL,0.00,2018,"[Free to Play, Indie, RPG, Strategy, Card Game..."
88312,670290,Real Pool 3D - Poolians,Poolians.com,0.00,2017,"[Casual, Free to Play, Indie, Simulation, Spor..."
88313,767400,弹炸人2222,彼岸领域,0.99,2017,"[Action, Adventure, Casual, Single-player]"
88315,772540,Battle Royale Trainer,Trickjump Games Ltd,3.99,2018,"[Action, Adventure, Simulation, FPS, Shooter, ..."


A partir de estos Dataframe, analizo lo que me piden:
- A partir del id de un usuario, devuelve la cantidad de dinero gastado por el usuario, el porcentaje de recomendación, y la cantidad de items que tiene el usuario. Emtonces lo que necesito es:
    - id de un usuario
    - cantidad de dinero gastado por el usuario
    - porcentaje de recomendación
    - cantidad de items que tiene el usuario

Veamos las columnas que tenemos disponibles en todos nuestros conjuntos de datos.

In [24]:
print(user_reviews.columns)
print()
print(user_items.columns)
print()
print(steam_games.columns)
print()

Index(['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend',
       'review', 'user_id'],
      dtype='object')

Index(['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'user_id'],
      dtype='object')

Index(['id', 'app_name', 'developer', 'price', 'release_year', 'categorical'], dtype='object')



De acuerdo a estos resultados...
Las columnas que voy a usar para este trabajo son:
- user_reviews: user_id, recommend
- user_items: user_id, item_id
- steam_games: id = item_id, price.

El resto de columnas no me sirve, así que las descarto

In [25]:
for column in user_reviews.columns: # Por cada columna en el dataframe user_reviews
    if column not in ['user_id', 'recommend']: # Si la columna no es la que me sirve
        user_reviews.drop(column, axis=1, inplace=True) # La elimino
print(user_reviews.columns) # Muestro las columnas que me quedaron cómo forma de comprobar los resultados

# Repito lo mismo con los otros dos dataframe

for column in user_items.columns:
    if column not in ['user_id', 'item_id']:
        user_items.drop(column, axis=1, inplace=True)
print(user_items.columns)

for column in steam_games.columns:
    if column not in ['price', 'id']:
        steam_games.drop(column, axis=1, inplace=True)
print(steam_games.columns)


Index(['recommend', 'user_id'], dtype='object')
Index(['item_id', 'user_id'], dtype='object')
Index(['id', 'price'], dtype='object')


Hechemos un vistazo a cómo quedaron los dataframe

In [32]:
user_reviews

,recommend,user_id
0,True,76561197970982479
1,True,76561197970982479
2,True,76561197970982479
3,True,js41637
4,True,js41637
...,...,...
59300,True,76561198312638244
59301,True,76561198312638244
59302,True,LydiaMorley
59303,True,LydiaMorley


In [33]:
user_items

,item_id,user_id
0,10,76561197970982479
1,20,76561197970982479
2,30,76561197970982479
3,40,76561197970982479
4,50,76561197970982479
...,...,...
5153204,346330,76561198329548331
5153205,373330,76561198329548331
5153206,388490,76561198329548331
5153207,521570,76561198329548331


In [34]:
steam_games

,id,price
88310,761140,4.99
88311,643980,0.00
88312,670290,0.00
88313,767400,0.99
88315,772540,3.99
...,...,...
120439,745400,1.99
120440,773640,1.99
120441,733530,4.99
120442,610660,1.99


Primero que nada, hago un chequeo de nulos en ambos 3 dataframes

In [35]:
user_items.isnull().sum() # Ninguno, perfecto

item_id    0
user_id    0
dtype: int64

In [36]:
user_reviews.isnull().sum() # Ninguno, perfecto

recommend    0
user_id      0
dtype: int64

In [37]:
steam_games.isnull().sum() # Perfecto

id       0
price    0
dtype: int64

Ahora que ya tengo los dataframe con las columnas que necesito, lo que me resulta más fácil de determinar es cuantos juegos tiene cada usuario. Para ello, utilizo el dataframe user_items

In [58]:
user_items[user_items["user_id"] == "76561197970982479"]["user_id"].count()

277

Primer prototipo del endpoint

In [59]:
def user_data(user_id):
    cantidad_items = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()
    return cantidad_items

In [60]:
user_data(76561197970982479) # La salida es la esperada

277

En base a esta cantidad de juegos que tiene el usuario, puedo determinar el porcentaje de recomendación del mismo.

In [63]:
user_reviews[(user_reviews["user_id"] == "76561197970982479") & user_reviews["recommend"] == True]["recommend"].count()

3

Segundo prototipo del endpoint

In [69]:
def user_data(user_id):
    cantidad_juegos = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()
    cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == str(user_id)) & user_reviews["recommend"] == True]["recommend"].count()
    return f"{round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)}% de Recomendación"

In [68]:
user_data(76561197970982479)

'1.08% de recomendación'

Ahora necesito encontrar la cantidad de dinero gastado por el usuario, para ello, tengo que usar los dataframe user_items y steam_games, ya que en ellos se encuentran todos los datos que necesito

In [74]:
steam_games[steam_games["id"] == 10]

,id,price
120416,10,9.99


In [83]:
user_items[user_items["item_id"] == 10] # Tengo que cambiarle el tipo de dato a la columna item_id

,item_id,user_id


In [85]:
user_items["item_id"] = user_items["item_id"].astype(int)

Me doy cuenta que lo más sencillo sería trabajar en un mismo dataframe, por lo cuál decido unir tanto el de items cómo el de games

In [87]:
usuario_juego_precio = pd.merge(user_items, steam_games, how="left", left_on="item_id", right_on="id")
usuario_juego_precio

,item_id,user_id,id,price
0,10,76561197970982479,10.0,9.99
1,20,76561197970982479,20.0,4.99
2,30,76561197970982479,30.0,4.99
3,40,76561197970982479,40.0,4.99
4,50,76561197970982479,50.0,4.99
...,...,...,...,...
5153204,346330,76561198329548331,346330.0,0.00
5153205,373330,76561198329548331,NaN,NaN
5153206,388490,76561198329548331,388490.0,0.00
5153207,521570,76561198329548331,521570.0,0.00


Voy a tomarme la libertad de eliminar las filas que contienen nulos, ya que de las mismas, no puedo asegurar que el juego es gratuito, ni tampoco que sea pago, así que las elimino

In [88]:
usuario_juego_precio.isnull().sum()

item_id          0
user_id          0
id         1064014
price      1064014
dtype: int64

In [90]:
usuario_juego_precio.dropna(inplace=True)
usuario_juego_precio

,item_id,user_id,id,price
0,10,76561197970982479,10.0,9.99
1,20,76561197970982479,20.0,4.99
2,30,76561197970982479,30.0,4.99
3,40,76561197970982479,40.0,4.99
4,50,76561197970982479,50.0,4.99
...,...,...,...,...
5153202,304930,76561198329548331,304930.0,0.00
5153203,227940,76561198329548331,227940.0,0.00
5153204,346330,76561198329548331,346330.0,0.00
5153206,388490,76561198329548331,388490.0,0.00


Filtremos el dataset usando el mismo usuario con el cuál veniamos haciendo las pruebas

In [94]:
usuario_juego_precio[usuario_juego_precio["user_id"] == "76561197970982479"]

,item_id,user_id,id,price
0,10,76561197970982479,10.0,9.99
1,20,76561197970982479,20.0,4.99
2,30,76561197970982479,30.0,4.99
3,40,76561197970982479,40.0,4.99
4,50,76561197970982479,50.0,4.99
...,...,...,...,...
272,289130,76561197970982479,289130.0,29.99
273,268500,76561197970982479,268500.0,59.99
274,461640,76561197970982479,461640.0,1.99
275,379720,76561197970982479,379720.0,29.99


De aquí, la columna que más nos interesa es la del precio, del cual queremos conocer la suma, la cantidad total gastada por el usuario, así que...

In [93]:
usuario_juego_precio[usuario_juego_precio["user_id"] == "76561197970982479"]["price"].sum() # Ese usuario gastó más de 3400 dólares

3404.3300000000004

Tercer prototipo del endpoint

In [105]:
def user_data(user_id):
    cantidad_juegos = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()
    cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == str(user_id)) & user_reviews["recommend"] == True]["recommend"].count()
    cantidad_dinero_gastado = usuario_juego_precio[usuario_juego_precio["user_id"] == str(user_id)]["price"].sum()
    return f"El usuario gastó {round(cantidad_dinero_gastado, 2)} USD"

In [106]:
user_data(76561197970982479)

'El usuario gastó 3404.33 USD'

Cuarto prototipo del endpoint

In [123]:
def user_data(user_id):
    diccionario_de_retorno = {}
    cantidad_juegos = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()
    cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == str(user_id)) & user_reviews["recommend"] == True]["recommend"].count()
    cantidad_dinero_gastado = usuario_juego_precio[usuario_juego_precio["user_id"] == str(user_id)]["price"].sum()

    diccionario_de_retorno["Usuario"] = user_id
    diccionario_de_retorno["Dinero gastado"] = f"{round(cantidad_dinero_gastado, 2)} USD"
    diccionario_de_retorno["Porcentaje de recomendación"] = f"{round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)}%"
    diccionario_de_retorno["Cantidad de juegos"] = cantidad_juegos
    
    return diccionario_de_retorno

In [124]:
user_data(76561197970982479)

{'Usuario': 76561197970982479,
 'Dinero gastado': '3404.33 USD',
 'Porcentaje de recomendación': '1.08%',
 'Cantidad de juegos': 277}

Bueno, se podría decir que ya resolví el endpoint 2, pero en realidad me faltan hacerle algunas aclaraciones y ajustes a la función para que su desempeño sea óptimo